In [1]:
from pathlib import Path
import numpy as np

In [2]:
input = Path("example.txt").read_text()
print(input)

MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX



In [3]:
lines = input.splitlines()
letter_map = {"X": 1, "M": 2, "A": 4, "S": 8}
X = np.array([[letter_map[letter] for letter in line] for line in lines])
X

array([[2, 2, 2, 8, 1, 1, 2, 4, 8, 2],
       [2, 8, 4, 2, 1, 2, 8, 2, 8, 4],
       [4, 2, 1, 8, 1, 2, 4, 4, 2, 2],
       [2, 8, 4, 2, 4, 8, 2, 8, 2, 1],
       [1, 2, 4, 8, 4, 2, 1, 4, 2, 2],
       [1, 1, 4, 2, 2, 1, 1, 4, 2, 4],
       [8, 2, 8, 2, 8, 4, 8, 1, 8, 8],
       [8, 4, 1, 4, 2, 4, 8, 4, 4, 4],
       [2, 4, 2, 2, 2, 1, 2, 2, 2, 2],
       [2, 1, 2, 1, 4, 1, 2, 4, 8, 1]])

In [4]:
X_pad = np.pad(X, 4)
X_pad

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 2, 2, 8, 1, 1, 2, 4, 8, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 8, 4, 2, 1, 2, 8, 2, 8, 4, 0, 0, 0, 0],
       [0, 0, 0, 0, 4, 2, 1, 8, 1, 2, 4, 4, 2, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 8, 4, 2, 4, 8, 2, 8, 2, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 2, 4, 8, 4, 2, 1, 4, 2, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 4, 2, 2, 1, 1, 4, 2, 4, 0, 0, 0, 0],
       [0, 0, 0, 0, 8, 2, 8, 2, 8, 4, 8, 1, 8, 8, 0, 0, 0, 0],
       [0, 0, 0, 0, 8, 4, 1, 4, 2, 4, 8, 4, 4, 4, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 4, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 1, 2, 1, 4, 1, 2, 4, 8, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
# Solution 1
total = 0
for direction in [(1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1)]:
    xmas = np.where(X_pad == 2**0, X_pad, 0)
    for i in range(1, 4):
        X_shift = np.where(X_pad == 2**i, X_pad, 0)
        X_shift = np.roll(X_shift, i * direction[0], axis=0)
        X_shift = np.roll(X_shift, i * direction[1], axis=1)
        xmas += X_shift
    total += (xmas == 15).sum()
total

18

In [6]:
# Solution 2
letter_map = {"X": 1, "M": 2, "A": 3, "S": 5}
X = np.array([[letter_map[letter] for letter in line] for line in lines])
X_pad = np.pad(X, 1, constant_values=1)
total = 0

xmas = np.where(X_pad == letter_map["A"], X_pad, 1)
for direction in (1, 1), (-1, -1), (-1, 1), (1, -1):
    for letter in ["S", "M"]:
        X_pre_shift = np.where(X_pad == letter_map[letter], X_pad, 1)

        X_shift = np.roll(X_pre_shift, direction[0], axis=0)
        X_shift = np.roll(X_shift, direction[1], axis=1)

        check_cross = np.roll(X_pre_shift, -direction[0], axis=0)
        check_cross = np.roll(check_cross, -direction[1], axis=1)

        X_shift = np.where(X_shift != check_cross, X_shift, 1)

        xmas *= X_shift
total = (xmas == (3 * 2 * 2 * 5 * 5)).sum()
# 2038 - too high
# 1992 - too high
# 1953 - too high
# 1929 - not right
print(total)

9


In [7]:
# Solution 2, using kernels
x_kernel = np.array(
    [
        [1, 0, 1],
        [0, 1, 0],
        [1, 0, 1],
    ]
)
kernels = np.array(
    [
        [
            [2, 0, 2],
            [0, 3, 0],
            [5, 0, 5],
        ],
        [
            [5, 0, 2],
            [0, 3, 0],
            [5, 0, 2],
        ],
        [
            [5, 0, 5],
            [0, 3, 0],
            [2, 0, 2],
        ],
        [
            [2, 0, 5],
            [0, 3, 0],
            [2, 0, 5],
        ],
    ]
)
x, y = X.shape
total = 0
for kernel in kernels:
    for i in range(x - 2):
        for j in range(y - 2):
            X_conv = X[i : i + 3, j : j + 3] * x_kernel
            if (X_conv == kernel).all():
                total += 1
total

9